In [251]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
from sklearn.model_selection import GridSearchCV

In [252]:
df = pd.read_csv("Data_Train_food.csv")

In [253]:
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [254]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w]', ' ', text)
    text = re.sub('[^A-Za-z0-9]+', '', text)
    return text

In [258]:
# df['Location'] = df['Location'].apply(clean_text)
# df['Cuisines'] = df['Cuisines'].apply(clean_text)

In [259]:
df['Restaurant']=le.fit_transform(df['Restaurant'])
df['Location']=le.fit_transform(df['Location'])
df['Cuisines']=le.fit_transform(df['Cuisines'])

In [260]:
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,5086,10,995,₹200,₹50,3.5,12,4,30 minutes
1,1785,30,1116,₹100,₹50,3.5,11,4,30 minutes
2,571,19,1182,₹150,₹50,3.6,99,30,65 minutes
3,4708,28,1398,₹250,₹99,3.7,176,95,30 minutes
4,4896,26,388,₹200,₹99,3.2,521,235,65 minutes


In [261]:
def price_convert(_price):
    return (re.sub(r'[^0-9.]', '', _price))

In [262]:
df['Average_Cost'] = df['Average_Cost'].apply(price_convert)
df['Minimum_Order'] = df['Minimum_Order'].apply(price_convert)
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,5086,10,995,200,50,3.5,12,4,30 minutes
1,1785,30,1116,100,50,3.5,11,4,30 minutes
2,571,19,1182,150,50,3.6,99,30,65 minutes
3,4708,28,1398,250,99,3.7,176,95,30 minutes
4,4896,26,388,200,99,3.2,521,235,65 minutes


In [263]:
trim_function = lambda x : re.findall("^\s*(.*?)\s*$",str(x))[0]

remove_commas = lambda x: re.sub("[^\d]", "", str(x))


df['Average_Cost']= df['Average_Cost'].apply(trim_function).apply(remove_commas)

In [264]:
#df['Average_Cost'] = df['Average_Cost'].astype(float)
df['Minimum_Order']=df['Minimum_Order'].astype(float)

In [265]:

df['Rating'] = df['Rating'].str.replace("-",'3.6')

In [266]:
df['Rating'].value_counts()

3.6                   2037
3.7                    869
3.5                    818
3.8                    800
NEW                    758
3.9                    749
3.4                    718
3.3                    675
4.0                    614
3.2                    511
4.1                    459
3.1                    411
3.0                    302
4.2                    272
4.3                    247
2.9                    199
2.8                    157
4.4                    142
4.5                     78
2.7                     76
2.6                     42
4.6                     41
4.7                     36
2.5                     27
2.4                     13
4.8                     13
Opening Soon            12
4.9                      8
2.3                      6
Temporarily Closed       2
2.1                      1
2.2                      1
Name: Rating, dtype: int64

In [267]:
df['Rating'] = df['Rating'].apply(lambda x: x.replace('NEW', '3.6'))
df['Rating'] = df['Rating'].apply(lambda x: x.replace('Opening Soon', '3.6'))
df['Rating'] = df['Rating'].apply(lambda x: x.replace('Temporarily Closed', '3.6'))

In [270]:
df['Votes'] = df['Votes'].apply(lambda x: x.replace('-', '244'))
df['Reviews'] = df['Reviews'].apply(lambda x: x.replace('-', '123'))

In [271]:
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,5086,10,995,200,50.0,3.5,12,4,30 minutes
1,1785,30,1116,100,50.0,3.5,11,4,30 minutes
2,571,19,1182,150,50.0,3.6,99,30,65 minutes
3,4708,28,1398,250,99.0,3.7,176,95,30 minutes
4,4896,26,388,200,99.0,3.2,521,235,65 minutes


In [272]:
df['Delivery_Time'] = df['Delivery_Time'].str.replace('minutes','').astype(float)

In [296]:
X = df.drop(['Average_Cost','Delivery_Time'],axis =1)
y = df['Delivery_Time']
y=y.astype(int)

In [297]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [298]:
from sklearn.svm import SVR
model = SVR(kernel='rbf')
model.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [276]:
y_pred = model.predict(X_test)

In [277]:
def rmlse(y_test, y_pred):
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    score = 1 - error
    return score

In [278]:
rmlse(y_test,y_pred)

0.8715205834151174

In [291]:
#random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
rf=RandomForestClassifier(n_estimators=10)

In [292]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [293]:
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7469358327325162